# Gather and Wrangle: We Rate Dogs Twitter Archive


**The goal of this project is to gather and clean data from the We Rate Dogs twitter acount. **

**The three datasets are:**
1. The twitter archive of WeRateDogs
2. Tweet image predictions of the dogs in the pictures according to a neural network
3. Each tweets favorite and retweet counts

**The three data sets will be downloaded in different ways:**
1. Twitter archive - Manually downloaded
2. Tweet image predictions-Programatically downloaded with the python requests library
3. Tweet favorite and retweet counts - Obtained from the Twitter API. 

Resources:
The tweet data dictionary is very helpful for deciphering the keys within the twitter JSON.
https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
https://stackoverflow.com/questions/7370801/measure-time-elapsed-in-python
https://wiki.python.org/moin/HandlingExceptions
https://gist.github.com/paulochf/f6c9ed0b39f85dd85270
https://stackoverflow.com/questions/6633678/finding-words-after-keyword-in-python

## 1. Gather the data. 
First load required packaged. Pandas for Pandas library, requests to load things programmatically with a url,
tweepy to access twitter API (this first needs to be installed in your ipython environment via pip install tweepy) and os to access parts of your operating system directory

In [505]:
import pandas as pd
import requests
import tweepy 
import os 
import json
from functools import reduce
#from load_api import load_api
import time
import re
from datetime import datetime



%matplotlib inline
%run load_api.py

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#expand column width to view the text output better
pd.set_option('max_colwidth', 150)


**First load the data that were manually downloaded. These data were obtained from the WeRateDogs twitter account managers who downloaded these data and made it available for students in the class.**

In [506]:
df_arch = pd.read_csv('Data/twitter-archive-enhanced.csv')
df_arch.head()
#df_arch.retweeted_status_id.value_counts()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https:/...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https:/...",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


In [507]:
df_arch.shape

(2356, 17)

**Now programmatically download the image prediction data.**

In [153]:
#url of where the data are
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

#now save the file to the computer
#put the data in the Data folder
folder_name = 'Data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)
    
#load them as a datafame
image_df = pd.read_csv('Data/image-predictions.tsv', sep='\t')
image_df.head()
image_df.shape

(2075, 12)

**Create API object to use to get the Twitter data**  
I have used a function to do this so as not to share my Twitter consumer key and access token.

In [65]:
api = load_api()

id_test = df_arch.tweet_id.iloc[0]
print(id_test)

#obtain tweet with info
tweet = api.get_status(id_test, tweet_mod='extended')

#entire JSON 
#print(tweet)
#tweet text
print(tweet.text)
#how many times the tweet has been retweeted
print(tweet.retweet_count)
#how many times the tweet has been favorited
print(tweet.favorite_count)
#how many followers the user has at the time of the tweet
print(tweet.user.followers_count)
#print(df_arch.tweet_id)

892420643555336193
This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU
8259
37853
7793827
0       892420643555336193
1       892177421306343426
2       891815181378084864
3       891689557279858688
4       891327558926688256
5       891087950875897856
6       890971913173991426
7       890729181411237888
8       890609185150312448
9       890240255349198849
10      890006608113172480
11      889880896479866881
12      889665388333682689
13      889638837579907072
14      889531135344209921
15      889278841981685760
16      888917238123831296
17      888804989199671297
18      888554962724278272
19      888202515573088257
20      888078434458587136
21      887705289381826560
22      887517139158093824
23      887473957103951883
24      887343217045368832
25      887101392804085760
26      886983233522544640
27      886736880519319552
28      886680336477933568
29      886366144734445568
               ...        
2326    6664115

**Query each tweet id and write its JSON data to the required tweet_json.txt file. Then read the txt file line by line to make a pandas DataFrame.**

Tips:

1. Try-except blocks may come in handy here.
2. Printing out each tweet ID after it was queried 
3. Using a code timer were both helpful 
4. Setting the wait_on_rate_limit and wait_on_rate_limit_notify parameters to True in the tweepy.api class is useful as well.



In [119]:
#loop through each tweet id in the dr_arch df


start = time.time()
print("starting")

data = []
tweet_errors = {}
for id in df_arch.tweet_id:
    try:
        tweet = api.get_status(id, tweet_mod='extended')
        print(tweet.id)
        date = tweet.created_at
        new_date = date.strftime('%Y-%m-%d %H-%M-%S')
        rt_ct = tweet.retweet_count
        fav_ct = tweet.favorite_count
        flr_ct = tweet.user.followers_count
        data.append({'id':tweet.id, 'date': new_date, 'retweet_cnt':rt_ct, "favorite_cnt": fav_ct, "followers":flr_ct})
        
        
    except Exception as e:
        print(str(tweet.id)+str(e))
        tweet_errors[tweet.id] = tweet.id
        

end = time.time()
print("total time elapsed = " + str(end - start))


starting
892420643555336193
892177421306343426
891815181378084864
891689557279858688
891327558926688256
891087950875897856
890971913173991426
890729181411237888
890609185150312448
890240255349198849
890006608113172480
889880896479866881
889665388333682689
889638837579907072
889531135344209921
889278841981685760
888917238123831296
888804989199671297
888554962724278272
888554962724278272[{'code': 144, 'message': 'No status found with that ID.'}]
888078434458587136
887705289381826560
887517139158093824
887473957103951883
887343217045368832
887101392804085760
886983233522544640
886736880519319552
886680336477933568
886366144734445568
886267009285017600
886258384151887873
886054160059072513
885984800019947520
885528943205470208
885518971528720385
885311592912609280
885167619883638784
884925521741709313
884876753390489601
884562892145688576
884441805382717440
884247878851493888
884162670584377345
883838122936631299
883482846933004288
883360690899218434
883117836046086144
882992080364220416
8

824775126675836928
824663926340194305
824325613288833024
824297048279236611
824025158776213504
823939628516474880
823719002937630720
823699002998870016
823581115634085888
823333489516937216
823322678127919110
823269594223824897
822975315408461824
822872901745569793
822859134160621569
822647212903690241
822610361945911296
822489057087389700
822462944365645825
822244816520155136
822163064745328640
821886076407029760
821813639212650496
821765923262631936
821522889702862852
821421320206483457
821407182352777218
821153421864615936
821149554670182400
821107785811234820
821044531881721856
820837357901512704
820749716845686786
820690176645140481
820494788566847489
820446719150292993
820314633777061888
820078625395449857
820013781606658049
819952236453363712
819924195358416896
819711362133872643
819588359383371776
819347104292290561
819238181065359361
819227688460238848
819015337530290176
819015331746349057
819006400881917954
819004803107983360
818646164899774465
818627210458333184
818614493328

770655142660169732
770414278348247044
770293558247038976
770093767776997377
770069151037685760
769940425801170949
769695466921623552
769335591808995329
769212283578875904
768970937022709760
768909767477751808
768855141948723200
768609597686943744
768596291618299904
768554158521745409
768473857036525572
768193404517830656
767884188863397888
767754930266464257
767500508068192258
767191397493538821
767122157629476866
766864461642756096
766793450729734144
766714921925144576
766693177336135680
766423258543644672
766313316352462849
766078092750233600
766069199026450432
766008592277377025
765719909049503744
765669560888528897
765395769549590528
765371061932261376
765222098633691136
764857477905154048
764259802650378240
763956972077010945
763837565564780549
763183847194451968
763167063695355904
763103485927849985
762699858130116608
762471784394268675
762464539388485633
762316489655476224
762035686371364864
761976711479193600
761750502866649088
761745352076779520
761672994376806400
761599872357

Rate limit reached. Sleeping for: 369


759446261539934208
759197388317847553
759159934323924993
759099523532779520
759047813560868866
758854675097526272
758828659922702336
758740312047005698
758474966123810816
758467244762497024
758405701903519748
758355060040593408
758099635764359168
758041019896193024
757741869644341248
757729163776290825
757725642876129280
757611664640446465
757597904299253760
757596066325864448
757400162377592832
757393109802180609
757354760399941633
756998049151549440
756939218950160384
756651752796094464
756526248105566208
756303284449767430
756288534030475264
756275833623502848
755955933503782912
755206590534418437
755110668769038337
754874841593970688
754856583969079297
754747087846248448
754482103782404096
754449512966619136
754120377874386944
754120377874386944[{'code': 144, 'message': 'No status found with that ID.'}]
753655901052166144
753420520834629632
753398408988139520
753375668877008896
753298634498793472
753294487569522689
753039830821511168
753026973505581056
752932432744185856
7529172845

706291001778950144
706265994973601792
706169069255446529
706166467411222528
706153300320784384
705975130514706432
705970349788291072
705898680587526145
705786532653883392
705591895322394625
705475953783398401
705442520700944385
705428427625635840
705239209544720384
705223444686888960
705102439679201280
705066031337840642
704871453724954624
704859558691414016
704847917308362754
704819833553219584
704761120771465216
704499785726889984
704491224099647488
704480331685040129
704364645503647744
704347321748819968
704134088924532736
704113298707505153
704054845121142784
703774238772166656
703769065844768768
703631701117943808
703611486317502464
703425003149250560
703407252292673536
703382836347330562
703356393781329922
703268521220972544
703079050210877440
703041949650034688
702932127499816960
702899151802126337
702684942141153280
702671118226825216
702598099714314240
702539513671897089
702332542343577600
702321140488925184
702276748847800320
702217446468493312
701981390485725185
701952816642

678969228704284672
678800283649069056
678798276842360832
678774928607469569
678767140346941444
678764513869611008
678755239630127104
678740035362037760
678708137298427904
678675843183484930
678643457146150913
678446151570427904
678424312106393600
678410210315247616
678399652199309312
678396796259975168
678389028614488064
678380236862578688
678341075375947776
678334497360859136
678278586130948096
678255464182861824
678023323247357953
678021115718029313
677961670166224897
677918531514703872
677895101218201600
677716515794329600
677700003327029250
677698403548192770
677687604918272002
677673981332312066
677662372920729601
677644091929329666
677573743309385728
677565715327688705
677557565589463040
677547928504967168
677530072887205888
677335745548390400
677334615166730240
677331501395156992


Rate limit reached. Sleeping for: 706


677328882937298944
677314812125323265
677301033169788928
677269281705472000
677228873407442944
677187300187611136
676975532580409345
676957860086095872
676949632774234114
676948236477857792
676946864479084545
676942428000112642
676936541936185344
676916996760600576
676897532954456065
676864501615042560
676821958043033607
676819651066732545
676811746707918848
676776431406465024
676617503762681856
676613908052996102
676606785097199616
676603393314578432
676593408224403456
676590572941893632
676588346097852417
676582956622721024
676575501977128964
676533798876651520
676496375194980353
676470639084101634
676440007570247681
676430933382295552
676263575653122048
676237365392908289
676219687039057920
676215927814406144
676191832485810177
676146341966438401
676121918416756736
676101918813499392
676098748976615425
676089483918516224
675898130735476737
675891555769696257
675888385639251968
675878199931371520
675870721063669760
675853064436391936
675849018447167488
675845657354215424
675822767435

668268907921326080
668256321989451776
668248472370458624
668237644992782336
668226093875376128
668221241640230912
668204964695683073
668190681446379520
668171859951755264
668154635664932864
668142349051129856
668113020489474048
667937095915278337
667924896115245057
667915453470232577
667911425562669056
667902449697558528
667886921285246976
667885044254572545
667878741721415682
667873844930215936
667866724293877760
667861340749471744
667832474953625600
667806454573760512
667801013445750784
667793409583771648
667782464991965184
667773195014021121
667766675769573376
667728196545200128
667724302356258817
667550904950915073
667550882905632768
667549055577362432
667546741521195010
667544320556335104
667538891197542400
667534815156183040
667530908589760512
667524857454854144
667517642048163840
667509364010450944
667502640335572993
667495797102141441
667491009379606528
667470559035432960
667455448082227200
667453023279554560
667443425659232256
667437278097252352
667435689202614272
667405339315

In [508]:
#serialize json data and save to text file
with open(os.path.join(folder_name,'json_data.txt'), mode='w') as outfile:  
    json.dump(data, outfile)

In [509]:
#convert the text file to a pandas data frame 
json_df = pd.DataFrame(data, columns=['id', 'date', 'retweet_cnt', 'favorite_cnt', 'followers'])
json_df.head()

,id,date,retweet_cnt,favorite_cnt,followers
0,892420643555336193,2017-08-01 16-23-56,8259,37853,7794034
1,892177421306343426,2017-08-01 00-17-27,6103,32526,7794034
2,891815181378084864,2017-07-31 00-18-03,4041,24490,7794034
3,891689557279858688,2017-07-30 15-58-51,8410,41206,7794034
4,891327558926688256,2017-07-29 16-00-24,9108,39388,7794034


In [510]:
json_df.shape

(2340, 5)

In [511]:
#check exceptions, there should be 16 of them which is the difference in the size between json_df and the df_arch
tweet_errors = list(tweet_errors)
tweet_errors
len(tweet_errors) == 16 
#ok, we are all set

True

### Join the dataframes together

Dataframes to join:
1. json_df  
2. df_arch 

Remove the tweet ids that were in the exception list


In [512]:
#change name of json_df column id
json_df = json_df.rename(columns={'id': 'tweet_id'})
json_df.head()



,tweet_id,date,retweet_cnt,favorite_cnt,followers
0,892420643555336193,2017-08-01 16-23-56,8259,37853,7794034
1,892177421306343426,2017-08-01 00-17-27,6103,32526,7794034
2,891815181378084864,2017-07-31 00-18-03,4041,24490,7794034
3,891689557279858688,2017-07-30 15-58-51,8410,41206,7794034
4,891327558926688256,2017-07-29 16-00-24,9108,39388,7794034


In [513]:
#join together and join on the right (json_df) so that it retains only the tweets that have a retweet and like count
new_df = df_arch.merge(json_df, on='tweet_id',how='right')

new_df.head()
new_df.info()
new_df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2339
Data columns (total 21 columns):
tweet_id                      2340 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2340 non-null object
source                        2340 non-null object
text                          2340 non-null object
retweeted_status_id           167 non-null float64
retweeted_status_user_id      167 non-null float64
retweeted_status_timestamp    167 non-null object
expanded_urls                 2281 non-null object
rating_numerator              2340 non-null int64
rating_denominator            2340 non-null int64
name                          2340 non-null object
doggo                         2340 non-null object
floofer                       2340 non-null object
pupper                        2340 non-null object
puppo                         2340 non-null object
date                          23

(2340, 21)

In [514]:
#now join the image predictions
df = new_df.merge(image_df, on='tweet_id')
df.info()
df.shape
#adding the image predictions reduced the tweets from 2340 to 2067

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2067 entries, 0 to 2066
Data columns (total 32 columns):
tweet_id                      2067 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2067 non-null object
source                        2067 non-null object
text                          2067 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2067 non-null object
rating_numerator              2067 non-null int64
rating_denominator            2067 non-null int64
name                          2067 non-null object
doggo                         2067 non-null object
floofer                       2067 non-null object
pupper                        2067 non-null object
puppo                         2067 non-null object
date                          2067 

(2067, 32)

## 2. Assess the data.

Look for issues specifically related to the ratings, the dog names, and dog stages. 

In [515]:
#now remove the ones that were retweets (retweeted status id) or replies (in_reply_to_status_id)
df1 = df.copy()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2067 entries, 0 to 2066
Data columns (total 32 columns):
tweet_id                      2067 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2067 non-null object
source                        2067 non-null object
text                          2067 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2067 non-null object
rating_numerator              2067 non-null int64
rating_denominator            2067 non-null int64
name                          2067 non-null object
doggo                         2067 non-null object
floofer                       2067 non-null object
pupper                        2067 non-null object
puppo                         2067 non-null object
date                          2067 

In [516]:
#df1[['text', 'name', 'rating_numerator', 'rating_denominator', 'doggo', 'floofer', 'pupper', 'puppo' ]].sample(20)
#np.set_printoptions(linewidth=400)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:500% !important; }</style>"))

pd.set_option('max_colwidth', 150)


df1[['text', 'name', 'rating_numerator', 'rating_denominator','doggo', 'floofer', 'pupper', 'puppo']].sample(20)

,text,name,rating_numerator,rating_denominator,doggo,floofer,pupper,puppo
1690,This is Fletcher. He's had a ruff night. No more Fireball for Fletcher. 8/10 it'll be over soon pupper https://t.co/tA4WpkI2cw,Fletcher,8,10,None,None,pupper,None
430,This is Lennon. He's a Boopershnoop Pupperdoop. Quite rare. Exceptionally pettable. 12/10 would definitely boop that shnoop https://t.co/fhgP6vSfhX,Lennon,12,10,None,None,None,None
1559,"""Yes hello I'ma just snag this here toasted bagel real quick. carry on."" 9/10 https://t.co/Cuz0Osnekp",None,9,10,None,None,None,None
1626,This pupper just wants a belly rub. This pupper has nothing to do w the tree being sideways now. 10/10 good pupper https://t.co/AyJ7Ohk71f,None,10,10,None,None,pupper,None
2046,This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,an,1,2,None,None,None,None
1268,This is Molly. She's a Peruvian Niddlewog. Loves her new hat. 11/10 would totally pet https://t.co/g4fiS8A9Ab,Molly,11,10,None,None,None,None
2017,These are Peruvian Feldspars. Their names are Cupit and Prencer. Both resemble Rand Paul. Sick outfits 10/10 &amp; 10/10 https://t.co/ZnEMHBsAs1,None,10,10,None,None,None,None
803,Meet Winston. He's pupset because I forgot to mention that it's Canada Day today. 11/10 please forgive me Winston https://t.co/xEY8dbJxnF,Winston,11,10,None,None,None,None
287,This is Alfie. He's your Lyft for tonight. Kindly requests you buckle pup and remain reasonably calm during the ride. 13/10 he must focus https://...,Alfie,13,10,None,None,None,None
1818,This is Bloo. He's a Westminster Cîroc. Doesn't think Bart deserves legs. Nice flowers. 8/10 https://t.co/IAc1QCczMc,Bloo,8,10,None,None,None,None


In [517]:
df1.name.value_counts()

None        575
a            55
Cooper       10
Lucy         10
Penny        10
Charlie      10
Tucker       10
Oliver       10
Lola          8
Bo            8
Winston       8
Sadie         8
Toby          7
Daisy         7
the           7
an            7
Scout         6
Bella         6
Bailey        6
Stanley       6
Rusty         6
Jax           6
Koda          6
Milo          6
Dave          6
Buddy         5
Oscar         5
Chester       5
Louis         5
Alfie         5
           ... 
Brockly       1
Bert          1
Malikai       1
Jeffri        1
Brooks        1
Monkey        1
Bruiser       1
Sailer        1
Huxley        1
Andru         1
Maya          1
Pablo         1
Dwight        1
Nollie        1
Asher         1
Chubbs        1
Blue          1
Arnold        1
Baron         1
Eugene        1
Laika         1
Ralph         1
Winifred      1
Suki          1
Crouton       1
Hall          1
Sky           1
Dug           1
Sonny         1
Samsom        1
Name: name, Length: 936,

In [232]:
df1.rating_numerator.value_counts()
df1.rating_denominator.value_counts()

10     2049
50        3
11        2
80        2
7         1
170       1
150       1
130       1
120       1
110       1
90        1
70        1
40        1
20        1
2         1
Name: rating_denominator, dtype: int64

In [247]:
df1[df1['name'] == 'None'][['text', 'name']].sample(20)

,text,name
1224,"This may be the greatest video I've ever been sent. 4/10 for Charles the puppy, 13/10 overall. (Vid by @stevenxx_) https://t.co/uaJmNgXR2P",None
108,RT @rachaeleasler: these @dog_rates hats are 13/10 bean approved https://t.co/nRCdq4g9gG,None
1327,"""You got any games on your phone"" 7/10 for invasive brown Dalmatian pupper https://t.co/yzGR9xjE9Q",None
1352,Everybody needs to read this. Jack is our first 14/10. Truly heroic pupper https://t.co/3m6bNGXWnM,None
1713,Lots of pups here. All are Judea Hazelnuts. Exceptionally portable. 8/10 for all https://t.co/Pa8EmpDCuI,None
1152,"I'm not sure what's happening here, but it's pretty spectacular. 12/10 for both https://t.co/JKXh0NbBNL",None
1511,"In honor of the new Star Wars movie. Here's Yoda pug. 12/10 pet really well, would I https://t.co/pvjdRn00XH",None
1590,What kind of person sends in a picture without a dog in it? 1/10 just because that's a nice table https://t.co/RDXCfk8hK0,None
900,*faints* 12/10 perfection in pupper form https://t.co/t6TxTwTLEK,None
1915,Fascinating dog here. Loves beach. Oddly long nose for dog. Massive ass paws. Hard to cuddle w. 3/10 would still pet https://t.co/IiSdmhkC5N,None


#### Dirty Data Issues:
1. Some retweets present (RT @ ...)
2. Some replies to other users present   
3. 55 names 'a'
4. 7 names 'an' 
5. 7 names 'the' 
6. Some none recordings when a name is available: Index 240: Name is recorded as None when name should be Atlas  , Index 1414: Name is recorded as None when should be Thea, Index 122, Index 129
7. Several rows with rating denominators wrong (962, 996, 1400, 2046, 871)  
8. 2 rows with rating denominator below 10 (likely mistake) 


#### Untidy Data Issues:
1. Dogger, floofer, pupper, and puppo seperate columns when should be in one column (stage). 
2. Date is combined into one column, should be separate. 



## Fix Issues

1. Remove the retweets by selecting only the null retweeted status ids

In [518]:
df1 = df1[df1.retweeted_status_id.isnull()]

In [519]:
#test
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1992 entries, 0 to 2066
Data columns (total 32 columns):
tweet_id                      1992 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     1992 non-null object
source                        1992 non-null object
text                          1992 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1992 non-null object
rating_numerator              1992 non-null int64
rating_denominator            1992 non-null int64
name                          1992 non-null object
doggo                         1992 non-null object
floofer                       1992 non-null object
pupper                        1992 non-null object
puppo                         1992 non-null object
date                          1992 non

2. Remove the replies by selecting only null in_reply_to_user_id

In [520]:
df1 = df1[df1.in_reply_to_user_id.isnull()]

In [521]:
#test
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 0 to 2066
Data columns (total 32 columns):
tweet_id                      1969 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1969 non-null object
source                        1969 non-null object
text                          1969 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1969 non-null object
rating_numerator              1969 non-null int64
rating_denominator            1969 non-null int64
name                          1969 non-null object
doggo                         1969 non-null object
floofer                       1969 non-null object
pupper                        1969 non-null object
puppo                         1969 non-null object
date                          1969 non-n

3. Fix the 55 names that were assigned 'a' by replacing the with their correct name. 

In [522]:
df1[df1['name'] =='a'][['text', 'name']]

,text,name
50,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/g...,a
519,Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,a
642,Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn,a
817,This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,a
819,Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R,a
828,This is a carrot. We only rate dogs. Please only send in dogs. You all really should know this by now ...11/10 https://t.co/9e48aPrBm2,a
854,This is a very rare Great Alaskan Bush Pupper. Hard to stumble upon without spooking. 12/10 would pet passionately https://t.co/xOBKCdpzaa,a
990,People please. This is a Deadly Mediterranean Plop T-Rex. We only rate dogs. Only send in dogs. Thanks you... 11/10 https://t.co/2ATDsgHD4n,a
1000,This is a taco. We only rate dogs. Please only send in dogs. Dogs are what we rate. Not tacos. Thank you... 10/10 https://t.co/cxl6xGY8B9,a
1117,Here is a heartbreaking scene of an incredible pupper being laid to rest. 10/10 RIP pupper https://t.co/81mvJ0rGRu,a


In [ ]:
#many of these have a name that follows the text 'named'- use this name or just replace with None. 

In [523]:
a_name = df1[df1['name'] == 'a']
a_name.head()


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
50,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004380872706/video/1,...,1,Samoyed,0.281463,True,Angora,0.272066,False,Persian_cat,0.114854,False
519,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.co...",...,4,web_site,0.226716,False,lighter,0.081941,False,switch,0.039009,False
642,772581559778025472,NaN,NaN,2016-09-04 23:46:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/772581559778025472/photo/1,https://twitter.com/dog_rates/status/772581559778025472/photo/1,https://twitter.co...",...,3,Newfoundland,0.574345,True,Border_collie,0.128352,True,Saint_Bernard,0.059476,True
817,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/747885874273214464/photo/1,https://twitter.com/dog_rates/status/747885874273214464/photo/1",...,1,kuvasz,0.408450,True,Samoyed,0.141330,True,pug,0.083018,True
819,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857231626240/photo/1,...,1,Pembroke,0.768923,True,Chihuahua,0.029053,True,Shetland_sheepdog,0.029035,True


In [524]:
a_new = a_name[a_name['text'].str.contains('name*')]
a_new

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1257,692187005137076224,NaN,NaN,2016-01-27 03:26:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a rare Arctic Wubberfloof. Unamused by the happenings. No longer has the appetites. 12/10 would totally hug https://t.co/krvbacIX0N,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/692187005137076224/photo/1,https://twitter.com/dog_rates/status/692187005137076224/photo/1,https://twitter.co...",...,2,Siberian_husky,0.810592,True,malamute,0.119745,True,Eskimo_dog,0.029265,True
1574,675706639471788032,NaN,NaN,2015-12-12 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675706639471788032/photo/1,...,1,English_springer,0.990300,True,Welsh_springer_spaniel,0.002080,True,cocker_spaniel,0.002014,True
1671,673636718965334016,NaN,NaN,2015-12-06 22:54:44 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Lofted Aphrodisiac Terrier named Kip. Big fan of bed n breakfasts. Fits perfectly. 10/10 would pet firmly https://t.co/gKlLpNzIl3,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673636718965334016/photo/1,...,1,wombat,0.880257,False,corn,0.019421,False,pug,0.019044,True
1750,671743150407421952,NaN,NaN,2015-12-01 17:30:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Tuscaloosa Alcatraz named Jacob (Yacōb). Loves to sit in swing. Stellar tongue. 11/10 look at his feet https://t.co/2IslQ8ZSc7,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671743150407421952/photo/1,...,1,toy_poodle,0.419427,True,miniature_poodle,0.237067,True,swing,0.104193,False
1781,671147085991960577,NaN,NaN,2015-11-30 02:01:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Helvetica Listerine named Rufus. This time Rufus will be ready for the UPS guy. He'll never expect it 9/10 https://t.co/34OhVhMkVr,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671147085991960577/photo/1,...,1,Yorkshire_terrier,0.467202,True,cairn,0.440122,True,silky_terrier,0.058690,True
1831,670427002554466305,NaN,NaN,2015-11-28 02:20:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute https://t.co/CtuJoLHiDo,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670427002554466305/photo/1,...,1,seat_belt,0.952258,False,toy_terrier,0.038872,True,beagle,0.003226,True
1840,670361874861563904,NaN,NaN,2015-11-27 22:01:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Rich Mahogany Seltzer named Cherokee. Just got destroyed by a snowball. Isn't very happy about it. 9/10 https://t.co/98ZBi6o4dj,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670361874861563904/photo/1,...,1,platypus,0.974075,False,spotted_salamander,0.011068,False,bison,0.003897,False
1843,670303360680108032,NaN,NaN,2015-11-27 18:09:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Speckled Cauliflower Yosemite named Hemry. He's terrified of intruder dog. Not one bit comfortable. 9/10 https://t.co/yV3Qgjh8iN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670303360680108032/photo/1,...,1,Shetland_sheepdog,0.380278,True,Cardigan,0.342806,True,guinea_pig,0.156249,False
1861,669923323644657664,NaN,NaN,2015-11-26 16:59:01 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nof

In [536]:
#new = a_new['text'].str.partition('named')[2].str.split('.')[0:]

new_names = []
for index, row in a_new.iterrows():
    
    string = str(row['text'])
    befor_keyowrd, keyword, after_keyword = string.partition('named')
    name = after_keyword.split('.')[0]
    new_names.append({'index': index, 'name': name})
    


new_names = pd.DataFrame(new_names)
new_names = new_names.set_index('index')
new_names


,name
index,
1257,
1574,Wylie
1671,Kip
1750,Jacob (Yacōb)
1781,Rufus
1831,Spork
1840,Cherokee
1843,Hemry
1861,Alphred


Now merge this to big dataset

In [537]:
df1.update(new_names)


In [538]:
#test this
df1[df1['name'] == 'a']
#works

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
50,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004380872706/video/1,...,1,Samoyed,0.281463,True,Angora,0.272066,False,Persian_cat,0.114854,False
519,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.co...",...,4,web_site,0.226716,False,lighter,0.081941,False,switch,0.039009,False
642,772581559778025472,NaN,NaN,2016-09-04 23:46:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/772581559778025472/photo/1,https://twitter.com/dog_rates/status/772581559778025472/photo/1,https://twitter.co...",...,3,Newfoundland,0.574345,True,Border_collie,0.128352,True,Saint_Bernard,0.059476,True
817,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/747885874273214464/photo/1,https://twitter.com/dog_rates/status/747885874273214464/photo/1",...,1,kuvasz,0.408450,True,Samoyed,0.141330,True,pug,0.083018,True
819,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857231626240/photo/1,...,1,Pembroke,0.768923,True,Chihuahua,0.029053,True,Shetland_sheepdog,0.029035,True
828,746872823977771008,NaN,NaN,2016-06-26 01:08:52 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a carrot. We only rate dogs. Please only send in dogs. You all really should know this by now ...11/10 https://t.co/9e48aPrBm2,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/746872823977771008/photo/1,https://twitter.com/dog_rates/status/746872823977771008/photo/1",...,1,Pembroke,0.540201,True,beagle,0.207835,True,Italian_greyhound,0.043565,True
854,743222593470234624,NaN,NaN,2016-06-15 23:24:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a very rare Great Alaskan Bush Pupper. Hard to stumble upon without spooking. 12/10 would pet passionately https://t.co/xOBKCdpzaa,NaN,NaN,NaN,https://twitter.com/dog_rates/status/743222593470234624/photo/1,...,1,kuvasz,0.350629,True,soft-coated_wheaten_terrier,0.182782,True,golden_retriever,0.087662,True
990,717537687239008257,NaN,NaN,2016-04-06 02:21:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",People please. This is a Deadly Mediterranean Plop T-Rex. We only rate dogs. Only send in dogs. Thanks you... 11/10 https://t.co/2ATDsgHD4n,NaN,NaN,NaN,https://twitter.com/dog_rates/status/717537687239008257/photo/1,..

Replace the remaining entries where names = 'a' to name = "None"

In [540]:
df1[df1['name'] == 'a'] = 'None'

In [542]:
#test this
df1[df1['name'] == 'a']
df1.name.value_counts()
#WORKS!

None        555
Cooper       10
Charlie      10
Lucy         10
Penny         9
Tucker        9
Oliver        9
Sadie         8
Winston       8
the           7
Toby          7
Lola          7
Daisy         7
Bella         6
Bo            6
Stanley       6
an            6
Leo           5
Buddy         5
Jax           5
Bailey        5
Oscar         5
Rusty         5
Louis         5
Dave          5
Scout         5
Koda          5
Milo          5
Chester       5
Gus           4
           ... 
Pete          1
Simba         1
Edmund        1
Butters       1
Sky           1
Crouton       1
Tassy         1
Suki          1
Cermet        1
Lilly         1
Kollin        1
Patch         1
Mingus        1
Skittles      1
Andru         1
Maya          1
Pablo         1
Dwight        1
Nollie        1
Asher         1
Chubbs        1
Blue          1
Arnold        1
Chase         1
Huxley        1
Baron         1
Laika         1
Ralph         1
Winifred      1
Tommy         1
Name: name, Length: 948,

4. Fix names with 'an'

In [545]:
df1.loc[1917, 'name'] = 'Berta'
df1[df1['name'] == 'an'] = 'None'

In [546]:
#test
df1[df1['name'] == 'an']
#works

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


5. Fix names with 'the'

In [548]:
df1[df1['name'] == 'the']

,text,name
1281,Stop sending in lobsters. This is the final warning. We only rate dogs. Thank you... 9/10 https://t.co/B9ZXXKJYNx,the
1347,This is the newly formed pupper a capella group. They're just starting out but I see tons of potential. 8/10 for all https://t.co/wbAcvFoNtn,the
1523,This is the happiest pupper I've ever seen. 10/10 would trade lives with https://t.co/ep8ATEJwRb,the
1540,This is the saddest/sweetest/best picture I've been sent. 12/10 😢🐶 https://t.co/vQ2Lw1BLBF,the
1753,This is the best thing I've ever seen so spread it like wildfire &amp; maybe we'll find the genius who created it. 13/10 https://t.co/q6RsuOVYwU,the
2056,This is the happiest dog you will ever see. Very committed owner. Nice couch. 10/10 https://t.co/RhUEAloehK,the
2057,Here is the Rand Paul of retrievers folks! He's probably good at poker. Can drink beer (lol rad). 8/10 good dog https://t.co/pYAJkAe76p,the


In [549]:
df1[df1['name'] == 'the'] = 'None'

In [550]:
#test
df1[df1['name'] == 'the']
#works

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


6. Replace names:  Name is recorded as None when name should be Atlas  , Index 1414: Name is recorded as None when should be Thea, Index 122, Index 129


In [557]:
df1.loc[1414]
df1.loc[1414, 'name'] = 'Thea'

In [562]:
df1.loc[129]
df1.loc[129, 'name'] = 'Boomer'